In [1]:
%cd /Users/yan/code/cnn-classifier/corpus

/Users/yan/Code/cnn-classifier/corpus


In [94]:
%pwd

u'/Users/yan/Code/cnn-classifier/corpus'

In [104]:
import argparse
import sys, os
import numpy as np
import pandas
from sklearn import metrics
import tensorflow as tf
from tensorflow.contrib import learn
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
FLAGS = None
MAX_LENGTH = 250
EMBEDDING_SIZE = 128
MAX_LABEL = 47
WORDS_FEATURE = 'words' 
FEATURE_LEN = 'words_len' 
BATCH_SIZE = 16
from torch.nn.utils.rnn import PackedSequence,pack_padded_sequence

In [16]:
corpus = open('questions_v2.tsv','r').readlines()

In [8]:
len(corpus)

235000

In [50]:
MIN_LENGTH=10
MAX_LENGTH=250
def parse_corpus(corpus):    
    X_r = []
    y_r = []

    for parallel in corpus:
        question, label = parallel[:-1].split('\t')
        if question.strip()=="" or label.strip()=="": continue
        X_r.append(question)
        y_r.append(label)
    return X_r, y_r

X_r, y_r = parse_corpus(corpus)
print(len(X_r),len(y_r))
print(X_r[0],y_r[0])

(234999, 234999)
('cancopsbegivenliedetectortest theonlywitnessisthespouse help', '55')


In [40]:
idx_pa = dict([(i, y) for i, y in enumerate(set(y_r))])
pa_idx = dict(zip(idx_pa.values(), idx_pa.keys()))

In [125]:
X_train, X_test, y_train, y_test = train_test_split(X_r, y_r, test_size=0.1, stratify=y_r, random_state=42)

In [18]:
def build_vocab():
    vocab = learn.preprocessing.VocabularyProcessor(MAX_LENGTH)

    for question in open('questions_v2.tsv'):
        body, _ = question.split('\t')
        tokens = [i for i in learn.preprocessing.tokenizer([body])][0]
        for token in tokens:
            vocab.vocabulary_.add(token)

    vocab.save('vocab')
    
def get_vocab():
    if not os.path.exists('vocab'):
        build_vocab()
    return learn.preprocessing.VocabularyProcessor.restore('vocab')   

vocab = get_vocab()

In [53]:
x_transform_train = vocab.fit_transform(X_train)
x_transform_test = vocab.fit_transform(X_test)

In [56]:
x_train = np.array(list(x_transform_train))
x_test = np.array(list(x_transform_test))
x_train_len = np.array([np.where(i>0)[0].shape[0] for i in x_train])
x_test_len = np.array([np.where(i>0)[0].shape[0] for i in x_test])

In [138]:
#{k:v for k,v in idx_pa.items()[:5]}
y_train_idx = np.array([pa_idx[int(i)] for i in y_train])
y_test_idx = np.array([pa_idx[int(i)] for i in y_test])

In [233]:
# only questions with more than 10 words are used in training
x_train_len_10_plus = np.where(x_train_len>10)[0]
x_train = x_train[x_train_len_10_plus]
x_train_len = x_train_len[x_train_len_10_plus] 
y_train_idx = y_train_idx[x_train_len_10_plus]

In [238]:
n_words = len(vocab.vocabulary_)
x_train.shape,y_train_idx.shape, n_words

((211337, 250), (211337,), 112332)

In [242]:
def estimator_spec_for_softmax_classification(
    logits, labels, mode):
  """Returns EstimatorSpec instance for softmax classification."""
  predicted_classes = tf.argmax(logits, 1)
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions={
            'class': predicted_classes,
            'prob': tf.nn.softmax(logits)
        })

  onehot_labels = tf.one_hot(labels, MAX_LABEL, 1, 0)
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

  eval_metric_ops = {
      'accuracy': tf.metrics.accuracy(
          labels=labels, predictions=predicted_classes)
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [193]:
def rnn_model(features, labels, mode):
  """RNN model to predict from sequence of words to a class."""
  # Convert indexes of words into embeddings.
  # This creates embeddings matrix of [n_words, EMBEDDING_SIZE] and then
  # maps word indexes of the sequence into [batch_size, sequence_length,
  # EMBEDDING_SIZE].
  word_vectors = tf.contrib.layers.embed_sequence(
      features[WORDS_FEATURE], vocab_size=n_words, embed_dim=EMBEDDING_SIZE)

  # Split into list of embedding per word, while removing doc length dim.
  # word_list results to be a list of tensors [batch_size, EMBEDDING_SIZE].
  #word_list = tf.unstack(word_vectors, axis=1)

  cell_fw = tf.nn.rnn_cell.GRUCell(num_units=EMBEDDING_SIZE)
  cell_bw = tf.nn.rnn_cell.GRUCell(num_units=EMBEDDING_SIZE)

  outputs, encoding = tf.nn.bidirectional_dynamic_rnn(
        cell_fw,
        cell_bw,
        dtype=tf.float32,
        sequence_length=features[FEATURE_LEN],
        inputs=word_vectors)
  '''
  cell = tf.nn.rnn_cell.GRUCell(EMBEDDING_SIZE)

  # Create an unrolled Recurrent Neural Networks to length of
  # MAX_DOCUMENT_LENGTH and passes word_list as inputs for each unit.
  _, encoding = tf.nn.static_rnn(cell, word_list, dtype=tf.float32)  '''

  # Given encoding of RNN, take encoding of last step (e.g hidden size of the
  # neural network of last step) and pass it as features for softmax
  # classification over output classes.
  cell_fw_state, cell_bw_state = encoding
  encoding = tf.concat(axis=1, values=[cell_fw_state, cell_bw_state])
  logits = tf.layers.dense(encoding, MAX_LABEL, activation=None)
  return estimator_spec_for_softmax_classification(
      logits=logits, labels=labels, mode=mode)

In [200]:
word_vectors = tf.contrib.layers.embed_sequence(x_train[:5], vocab_size=n_words, embed_dim=EMBEDDING_SIZE)
cell_fw = tf.nn.rnn_cell.GRUCell(num_units=EMBEDDING_SIZE)
cell_bw = tf.nn.rnn_cell.GRUCell(num_units=EMBEDDING_SIZE)
outputs, encoding = tf.nn.bidirectional_dynamic_rnn(
    cell_fw,
    cell_bw,
    dtype=tf.float32,
    sequence_length=x_train_len[:5],
    inputs=word_vectors)
cell_fw_state, cell_bw_state = encoding
encoding = tf.concat(axis=1, values=[cell_fw_state, cell_bw_state])
encoding.get_shape()

TensorShape([Dimension(None), Dimension(256)])

In [243]:
#tf.reset_default_graph()
model_fn = rnn_model

train_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={WORDS_FEATURE: x_train, FEATURE_LEN: x_train_len}, 
  y=y_train_idx,
  batch_size=BATCH_SIZE,
  num_epochs=5,
  shuffle=True)

classifier = tf.estimator.Estimator(model_fn=model_fn, model_dir="/Users/yan/code/cnn-classifier/corpus/log_birnn/")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_model_dir': '/Users/yan/code/cnn-classifier/corpus/log_birnn/', '_save_summary_steps': 100}


In [248]:
classifier.train(input_fn=train_input_fn, steps=300)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /Users/yan/code/cnn-classifier/corpus/log_birnn/model.ckpt-3000
INFO:tensorflow:Saving checkpoints for 3001 into /Users/yan/code/cnn-classifier/corpus/log_birnn/model.ckpt.
INFO:tensorflow:loss = 1.28227, step = 3001
INFO:tensorflow:global_step/sec: 2.58049
INFO:tensorflow:loss = 2.15128, step = 3101 (38.754 sec)
INFO:tensorflow:global_step/sec: 2.62181
INFO:tensorflow:loss = 2.13417, step = 3201 (38.151 sec)
INFO:tensorflow:Saving checkpoints for 3300 into /Users/yan/code/cnn-classifier/corpus/log_birnn/model.ckpt.
INFO:tensorflow:Loss for final step: 1.28783.


In [249]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={WORDS_FEATURE: x_test, FEATURE_LEN: x_test_len},
      y=y_test_idx,
      num_epochs=1,
      batch_size=BATCH_SIZE,
      shuffle=False)

predictions = classifier.predict(input_fn=test_input_fn)


In [250]:
y_predicted = np.array(list(p['class'] for p in predictions))
y_predicted = y_predicted.reshape(np.array(y_test).shape)

# Score with sklearn.
score = metrics.accuracy_score(y_test_idx, y_predicted)
print('Accuracy (sklearn): {0:f}'.format(score))

# Score with tensorflow.
scores = classifier.evaluate(input_fn=test_input_fn)
print('Accuracy (tensorflow): {0:f}'.format(scores['accuracy']))

INFO:tensorflow:Restoring parameters from /Users/yan/code/cnn-classifier/corpus/log_birnn/model.ckpt-3300
Accuracy (sklearn): 0.580894
INFO:tensorflow:Starting evaluation at 2017-12-06-03:49:12
INFO:tensorflow:Restoring parameters from /Users/yan/code/cnn-classifier/corpus/log_birnn/model.ckpt-3300
INFO:tensorflow:Finished evaluation at 2017-12-06-03:50:52
INFO:tensorflow:Saving dict for global step 3300: accuracy = 0.580894, global_step = 3300, loss = 1.44655
Accuracy (tensorflow): 0.580894
